In [1]:
import sys,os
import csv
import pandas as pd

In [5]:
files = ['flex_data_2-6.xlsx', 
        'flex_data_6-9.xlsx',
        'scc_data_1-6.xlsx',
        'scc_data_6-9.xlsx']

alldata = pd.DataFrame()
for idx, val in enumerate(files):
    file = val
    print("Reading file ", file)
    data = pd.read_excel(file)
    #data.info()
    for idx, row in data.iterrows():
        row['MotherSN'] = row['MotherSN'].strip("\'")
    alldata = alldata.append(data)
    
alldata.info()

Reading file  flex_data_2-6.xlsx
Reading file  flex_data_6-9.xlsx
Reading file  scc_data_1-6.xlsx
Reading file  scc_data_6-9.xlsx
<class 'pandas.core.frame.DataFrame'>
Int64Index: 97539 entries, 0 to 25286
Data columns (total 2 columns):
MotherSN    97539 non-null object
ChildSN     97486 non-null object
dtypes: object(2)
memory usage: 2.2+ MB


In [6]:
for i in range(10):
    mother = alldata.iloc[i]['MotherSN']
    child = alldata.iloc[i]['ChildSN']
    print("MotherSN = {}, ChildSN = {}".format(mother, child))

MotherSN = 10006244181R0004, ChildSN = XR-X097013AB801100990
MotherSN = 10006244181R0053, ChildSN = XR-X097013AB801100535
MotherSN = 10006244181R0114, ChildSN = 4555107GB1B1968
MotherSN = 10006244181R0118, ChildSN = SZGR201701623
MotherSN = 10006244181W0046, ChildSN = 110044181Q005BC0
MotherSN = 10006244181W0046, ChildSN = 120024181N0068C0
MotherSN = 10006244181W0046, ChildSN = 040014181N002QC0
MotherSN = 10006244181W0046, ChildSN = SZGR201701422
MotherSN = 10006244181W0046, ChildSN = 090024181S005FD0
MotherSN = 10006244181W0046, ChildSN = 000194181T0011A0


In [13]:
out_columns = ['PackDate', 'MotherSN', 'MB', 'NIBP', 'SPO2', 'GLU', 'ECG']
sndata = pd.read_excel("ProductSN_3-9.xlsx")
sndata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8647 entries, 0 to 8646
Data columns (total 2 columns):
ProductSN    8647 non-null object
PackDate     8647 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 135.2+ KB


In [18]:
print(len(sndata))
sndata.head()

8647


,ProductSN,PackDate
0,GT10006244181W0053,2018-03-05
1,GT1000624418210035,2018-03-05
2,GT1000624418210117,2018-03-05
3,GT1000624418210123,2018-03-05
4,GT1000624418210138,2018-03-05


In [24]:
%%timeit
outsndata = pd.DataFrame(columns=out_columns)
for i, p in sndata.iterrows():
    msn = p['ProductSN'][2:]
    childsn = alldata[alldata['MotherSN'] == msn]
    #childsn.info()
    for idx, row in childsn.iterrows():
        sn = str(row['ChildSN'])
        if (sn[:3] == '000'): 
            mbsn = sn[:16]
        elif (sn[:4] == '0400'):
            glusn = sn[:16]
        elif (sn[:4] == '2000' or sn[:4] == '1200'):
            nibpsn = sn[:16]
        elif (sn[:4] == '1100'):
            spo2sn = sn[:16]
        elif (sn[:4] == 'SZGR' or sn[:4] == 'JSGR'):
            ecgsn = sn[:13]
    #print("MotherSN:{}, MB:{}, GLU:{}, NIBP:{}, SPO2:{}, ECG:{}".format(msn, mbsn, glusn, nibpsn, spo2sn, ecgsn))
    s = pd.Series({'PackDate':p['PackDate'], 'MotherSN':msn, 'MB':mbsn, 'NIBP':nibpsn, 'SPO2':spo2sn, 'GLU':glusn, "ECG":ecgsn})
    outsndata = outsndata.append(s, ignore_index=True)
    
outsndata.info()
outsndata.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8647 entries, 0 to 8646
Data columns (total 7 columns):
PackDate    8647 non-null datetime64[ns]
MotherSN    8647 non-null object
MB          8647 non-null object
NIBP        8647 non-null object
SPO2        8647 non-null object
GLU         8647 non-null object
ECG         8647 non-null object
dtypes: datetime64[ns](1), object(6)
memory usage: 473.0+ KB


,PackDate,MotherSN,MB,NIBP,SPO2,GLU,ECG
0,2018-03-05,10006244181W0053,000194181T0002A0,120024181N004JC0,110044181Q002NC0,04001417CC006SC0,SZGR201701416
1,2018-03-05,1000624418210035,000194181W000WA0,120024181N002AC0,110044181Q0060C0,040014181N001KC0,SZGR201702255
2,2018-03-05,1000624418210117,000194181W000HA0,120024181W002AC0,110044181X0014C0,040014181V00L1C0,SZGR201702985
3,2018-03-05,1000624418210123,000194181W0025A0,120024181W0047C0,110044181X00D2C0,040014181V002PC0,SZGR201609873
4,2018-03-05,1000624418210138,000194181W004ZA0,120024181W006WC0,110044181X006JC0,040014181V0024C0,SZGR201702973


In [25]:
outsndata.to_excel("Product_Child_SN.xls")